# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic). Please, note, that you must not rely on any specific location for the dataset, hence, any code like

```python
titanic_train = pd.read_csv("<location>/train.csv")
```

will fail and your notebook won't be validated and graded. Inputs to the functions are described explicitly in each case, and that's the only thing you can rely on.

In [1]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [2]:
plt.rcParams["figure.figsize"] = (6,6)

In [3]:
import numpy as np
import pandas as pd

In [4]:
STUDENT = "Asaf Dahan"
ASSIGNMENT = 4
TEST = False

In [5]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 16

In [6]:
if not TEST:
    df_train = pd.read_csv('./train.csv')
    df_test = pd.read_csv('./test.csv')
    df = pd.concat([df_train, df_test])
    df.set_index('PassengerId', inplace=True)

# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must **replace the corresponding age value in the dataframe with her actual age at the time** (25) and return the 
dataset. Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return a copy of the input dataframe, and not perform replacement in the original dataframe. Structure and indexing must be the same as in input.

In [7]:
def fix_age(df):
    """Fix age for Julia Florence Siegel."""
    df.loc[df.Name.str.contains('Julia Florence Siegel'), ['Age']] = 26.
    return df

In [8]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. You must return **series**, indexed with values from `Embarked`, according to `.value_counts()` method semantics:

```
S    <number of passengers in 3-d class, embarked at S, 20<=Age<=30>
C    <...>
Q    <...>
Name: Embarked, dtype: int64
```

In [9]:
def embarked_stats(df):
    """Calculate embarkment port statistics."""
    return df[(df.Age >= 20.) & (df.Age <= 30.) & (df.Sex == 'male') & (df.Pclass == 3)][['Embarked']].value_counts()

In [10]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age over all passengers, and fill missing age values in `Age` column.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [11]:
def fix_age(df):
    """Fix missing age values."""
    df2 = df.copy()
    df2.loc[df2.Age.isnull(), ['Age']] = df2.Age.mean()
    return df2

In [12]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's **only one child** like this.

In [13]:
def get_nursemaid(df):
    filtered = df[(df.Age < 10) & (df.SibSp == 0) & (df.Parch == 0)]
    # By looking on the dataset I found that the nursemaid had the same ticket number
    merged = filtered.merge(df[['Name', 'Cabin', 'Ticket','SibSp', 'Age']], on='Ticket')
    res = merged[merged['Name_x'] != merged['Name_y']]
    return res.iloc[0].Name_x, res.iloc[0].Name_y 

In [14]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **single string** with port letter.

In [15]:
def get_port(df):
    """Get port with the most children embarked."""
    dfp = df.reset_index()
    kids = dfp[dfp.Age < 10].groupby(['Embarked']).PassengerId.nunique()
    adults = dfp[~(dfp.Age < 10)].groupby(['Embarked']).PassengerId.nunique()
    return (kids/adults * 100.).sort_values(ascending=False).reset_index().iloc[0].Embarked

In [16]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **tuple** of two values - average and maximum number of passengers per ticket.

In [17]:
def get_ticket_stats(df):
    """Calculate passenger per ticket statistics."""
    res = df.reset_index().groupby(['Ticket']).agg({'PassengerId': 'nunique'}).reset_index()
    return res.PassengerId.mean(), res.PassengerId.max()

In [18]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

### 7. Fare per passenger (3 points).

For each individual ticket, you must calculate **fare per person for that ticket**, and then calculate averages for each class. Note, that you will need to apply `groupby` and you may consider using `.first()` of resulting `DataFrameGroupBy`. Also, caferully consider, in which order calculations are performed.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `pd.Series` with three elements, indexed by class:

```
1    <average per person fare in class 1>
2    <...>
3    <...>
Name: Pclass, dtype: float64
```

In [19]:
def get_fare_per_pass(df):
    """Calculate fare per passenger for different classes."""
    pass_per_ticket = dfp.groupby(['Ticket', 'Pclass', 'Fare']).agg({'PassengerId': 'nunique'}).reset_index()
    per_class = pass_per_ticket.groupby(['Pclass']).agg({'Fare': 'sum', 'PassengerId': 'sum'})
    return pd.Series(per_class['Fare']/per_class['PassengerId'], name='Pclass')


In [20]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with global average over all passengers. Now, you need to fill them **according to class and sex**. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [21]:
def fix_age_groupped(df):
    """Fill missing age values."""
    df2 = df.copy()
    fixed_age = df2.groupby(['Pclass', 'Sex']).agg({'Age': 'mean'}).reset_index()
    res_df = df2.merge(fixed_age, on=['Pclass', 'Sex']).rename({'Age_x': 'Age', 'Age_y': 'classSexAge'}, axis=1)
    res_df.loc[res_df.Age.isnull(), ['Age']] = res_df.classSexAge
    return res_df.drop(['classSexAge'], axis=1)

In [22]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of the number couples with partial survival or couples which died together, divided by total number of couples in class. If the survival status of one or both of spouses is not known, it must be considered as `0`.

Input is **indexed** with `PassengerId` and is a **concatenation of train and test sets**. Output must be `Series` with three elements indexed by values from `Pclass` column (see P7 as a reference).

In [23]:
def find_couples(df):
    """Fill missing age values."""
    # Coupeling code from lecture 5
    family_names = (df
                    .replace(re.compile(r'\s+\(.*\)'), '')
                    .replace(re.compile("Mrs."), "Mr."))[["Name", "Sex"]]
    family_names = family_names[(family_names.Sex=="female") & family_names.Name.str.contains("Mr.")]
    couples = (df.join(family_names
                            .reset_index()
                            .set_index("Name")["PassengerId"],
                            on="Name", how="inner", rsuffix="_Spouse"))
    couples.rename({"PassengerId":"PassengerId_Spouse"},
                   axis=1, inplace=True)
    couples = couples.join(df[["Name", "Survived"]],
                           on="PassengerId_Spouse", rsuffix="_Spouse")
    
    # Fillna survived missing values with 0.
    couples.Survived.fillna(0., inplace=True)
    couples.Survived_Spouse.fillna(0., inplace=True)
    # Create survived class (0 - Both dead | 1 - One surviver | 2 - Both survived)
    couples['Survived_class'] = couples.Survived + couples.Survived_Spouse
    return pd.Series(couples[couples.Survived_class.isin([0., 1.])].groupby('Pclass').size() / couples.groupby('Pclass').size(), name='Pclass')

In [24]:
PROBLEM_ID = 9

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)

In [25]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")